In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_rnn_2025_04_03_11_14.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,graph_powerlaw_cluster_graph_n7,3.0,2.935704
1,graph_powerlaw_cluster_graph_n7,0.0,0.000000
2,graph_powerlaw_cluster_graph_n7,2.0,2.006102
3,graph_powerlaw_cluster_graph_n7,0.0,0.000000
4,graph_powerlaw_cluster_graph_n7,1.0,0.960452


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,graph_powerlaw_cluster_graph_n7,3.0,2.935704,0.064296,0.004134,0.0,0.021432
1,graph_powerlaw_cluster_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
2,graph_powerlaw_cluster_graph_n7,2.0,2.006102,0.006102,0.000037,0.0,0.003051
3,graph_powerlaw_cluster_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,graph_powerlaw_cluster_graph_n7,1.0,0.960452,0.039548,0.001564,0.0,0.039548
...,...,...,...,...,...,...,...
5275,star_graph_n7,0.0,0.419709,0.419709,0.176156,1.0,0.419709
5276,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
5277,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
5278,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000


In [9]:
df['RE'].sum()/len(df)

0.04739176189398153

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1781,119.039417,0.066839,0.104036
1,2.0,1638,44.844434,0.027378,0.027275
2,1.0,902,33.999618,0.037694,0.002186
3,4.0,670,38.823208,0.057945,0.104711
4,0.0,223,1.234290,0.005535,0.001207
5,5.0,65,11.994132,0.184525,0.888420
6,6.0,1,0.293403,0.293403,3.099081


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,183.220679,0.046895,0.072148
1,graph_powerlaw_cluster_graph_n7,1350,65.560899,0.048564,0.059546
2,star_graph_n7,23,1.446925,0.062910,0.040854


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3797,182.723318,0.048123,0.074222
1,graph_powerlaw_cluster_graph_n7,1258,65.357794,0.051954,0.063887
2,star_graph_n7,2,0.913101,0.456551,0.375235


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,442,16.993252,0.038446,0.036592
1,graph_powerlaw_cluster_graph_n7,3.0,346,26.992924,0.078014,0.111981
2,graph_powerlaw_cluster_graph_n7,1.0,338,12.306339,0.036409,0.001431
3,graph_powerlaw_cluster_graph_n7,4.0,115,6.018126,0.052332,0.084387
4,graph_powerlaw_cluster_graph_n7,0.0,92,0.203104,0.002208,0.000190
5,graph_powerlaw_cluster_graph_n7,5.0,16,2.753750,0.172109,0.760212
6,graph_powerlaw_cluster_graph_n7,6.0,1,0.293403,0.293403,3.099081
7,graph_random_regular_graph_n7_d4,3.0,1434,91.958194,0.064127,0.102143
8,graph_random_regular_graph_n7_d4,2.0,1196,27.851183,0.023287,0.023832
9,graph_random_regular_graph_n7_d4,1.0,563,20.868476,0.037067,0.001435
